In [59]:
cd C:/Users/karit/Documents/fin

C:\Users\karit\Documents\fin


In [12]:
fp = "Vaestotietoruudukko_2015.shp"

In [14]:
import geopandas as gpd

In [16]:
pop = gpd.read_file(fp)

In [17]:
# See the first rows
In [1]: pop.head()

,ASUKKAITA,ASVALJYYS,IKA0_9,IKA10_19,IKA20_29,IKA30_39,IKA40_49,IKA50_59,IKA60_69,IKA70_79,IKA_YLI80,INDEX,geometry
0,8,31.0,99,99,99,99,99,99,99,99,99,688,"POLYGON ((25472499.99532626 6689749.005069185,..."
1,6,42.0,99,99,99,99,99,99,99,99,99,703,"POLYGON ((25472499.99532626 6685998.998064222,..."
2,8,44.0,99,99,99,99,99,99,99,99,99,710,"POLYGON ((25472499.99532626 6684249.004130407,..."
3,7,64.0,99,99,99,99,99,99,99,99,99,711,"POLYGON ((25472499.99532626 6683999.004997005,..."
4,19,23.0,99,99,99,99,99,99,99,99,99,715,"POLYGON ((25472499.99532626 6682998.998461431,..."


In [18]:
# Change the name of a column
In [2]: pop = pop.rename(columns={'ASUKKAITA': 'pop15'})

In [19]:
# See the column names and confirm that we now have a column called 'pop15'
In [3]: pop.columns

Index([    u'pop15', u'ASVALJYYS',    u'IKA0_9',  u'IKA10_19',  u'IKA20_29',
        u'IKA30_39',  u'IKA40_49',  u'IKA50_59',  u'IKA60_69',  u'IKA70_79',
       u'IKA_YLI80',     u'INDEX',  u'geometry'],
      dtype='object')

In [21]:
# Columns that will be sected
In [4]: selected_cols = ['pop15', 'geometry']

In [22]:
# Select those columns
In [5]: pop = pop[selected_cols]

In [23]:
# Let's see the last 2 rows
In [6]: pop.tail(2)

,pop15,geometry
5782,9,"POLYGON ((25513499.99632164 6685498.999797418,..."
5783,30244,"POLYGON ((25513999.999929 6659998.998172711, 2..."


In [37]:
import fiona

In [49]:
# Addresses filpath
In [7]: addr_fp = r"/Users/karit/Documents/fin/addresses_epsg3879.shp"

In [50]:
# Read data
In [8]: addresses = gpd.read_file(addr_fp)

In [51]:
# Check the head of the file
In [9]: addresses.head(2)

,address,geometry,id
0,"Kampinkuja 1, 00100 Helsinki, Finland",POINT (25496123.30852197 6672833.941567578),1001
1,"Kaivokatu 8, 00101 Helsinki, Finland",POINT (25496774.28242895 6672999.698581985),1002


In [52]:
# Check the crs of address points
In [10]: addresses.crs

{u'ellps': u'GRS80',
 u'k': 1,
 u'lat_0': 0,
 u'lon_0': 25,
 u'no_defs': True,
 u'proj': u'tmerc',
 u'units': u'm',
 u'x_0': 25500000,
 u'y_0': 0}

In [53]:
# Check the crs of population layer
In [11]: pop.crs

{u'ellps': u'GRS80',
 u'k': 1,
 u'lat_0': 0,
 u'lon_0': 25,
 u'no_defs': True,
 u'proj': u'tmerc',
 u'units': u'm',
 u'x_0': 25500000,
 u'y_0': 0}

In [54]:
# Do they match? - We can test that
In [12]: addresses.crs == pop.crs

True

In [55]:
# Make a spatial join
In [13]: join = gpd.sjoin(addresses, pop, how="inner", op="within")

In [56]:
# Let's check the result
In [14]: join.head()

,address,geometry,id,index_right,pop15
0,"Kampinkuja 1, 00100 Helsinki, Finland",POINT (25496123.30852197 6672833.941567578),1001,3326,173
1,"Kaivokatu 8, 00101 Helsinki, Finland",POINT (25496774.28242895 6672999.698581985),1002,3449,31
10,"Rautatientori 1, 00100 Helsinki, Finland",POINT (25496808.64582102 6673146.836896984),1011,3449,31
3,"Itäväylä, 00900 Helsinki, Finland",POINT (25505098.34340289 6677972.568484427),1004,5112,353
4,"Tyynenmerenkatu 9, 00220 Helsinki, Finland",POINT (25495639.56049686 6671520.343245601),1005,3259,1397


In [57]:
# Output path
outfp = r"/Users/karit/Documents/fin/addresses_pop15_epsg3979.shp"

In [58]:
# Save to disk
join.to_file(outfp)